## Project: Analysis and Classification of customer churn in telecommunication companies

### Author: Munezero Mihigo

### Date: 13 November 2021

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.stats.mstats import gmean
import seaborn as sns
import matplotlib.pyplot as plt